In [ ]:
import requests
import time
from bs4 import BeautifulSoup
from lxml import etree
import csv
import pandas as pd
import pymongo
import json
import threading

#定义抓取的房屋信息的dataframe，以及dataframe的index
host_info=pd.DataFrame(columns=["小区名称","房子标题","朝向", "交易日期", "成交总价", "楼层", "单价", "满几信息", "挂牌总价", "交易周期"])
host_index = 0 


#定义多线程的锁
lock = threading.Lock()

In [ ]:
headers={
    'Host': 'bj.lianjia.com',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'Accept': '*/*',
    'Referer': 'https://bj.lianjia.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9'
}


In [ ]:
def funds_f_mongo(colname):
    myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
    mydb = myclient["Lianjia"]
    mycol = mydb[colname]
    temp = []
    for x in mycol.find({},{"_id":0}):
        temp.append(x)
    xiaoqu = pd.read_json(json.dumps(temp))
    return xiaoqu

def funds_2_mongo(colname,df1):
    #将pd转换成了字典，然后写入mongo
    tmp_funds2 = df1.to_dict(orient='records')

    myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
    mydb = myclient["Lianjia"]
    mycol = mydb[colname]
    mycol.drop()
    mycol.insert_many(tmp_funds2)

In [ ]:
def get_xiaoqu2():
    head = {'Host': 'bj.lianjia.com',
            'Referer': 'https://bj.lianjia.com/chengjiao/',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'
            }

    url = 'https://bj.lianjia.com/xiaoqu/?from=rec'
    res = requests.get(url=url,headers=head)
    content=etree.HTML(res.text)
    xiaoqu_number=content.xpath('/html/body/div[4]/div[1]/div[2]/h2/span/text()')[0]
    print(xiaoqu_number)
    #xiaoqu_pages=int(xiaoqu_number)/30
    xiaoqu_id = []
    xiaoqu_name = []
    fail_urls = []
    for i in range(1,101):
        url = 'https://bj.lianjia.com/xiaoqu/pg' + str(i) + '/?from=rec' 
        print(url)
        try:
            #r = requests.get(url, headers=head, proxies=p, timeout=3)
            r = requests.get(url, headers=head, timeout=3)
            html = etree.HTML(r.text)
            datas=html.xpath('//li[@class="clear xiaoquListItem"]/@data-id')
            titles=html.xpath('//li[@class="clear xiaoquListItem"]/div[@class="info"]/div[@class="title"]/a/text()')
            #print('No:' + str(x), 'page:' + str(i), len(s), len(datas), len(title))
            #如果当前页没有返回数据，将当前页数加到列表l末尾，再次抓取
            if len(datas)==0:
                print(url)
                fail_urls.append(i)
            else:
                for data in datas:
                    print(data)
                    xiaoqu_id.append(data)
                    
                for title in titles:
                    print(title)
                    xiaoqu_name.append(title)
        # 如果当前页访问出现异常，将当前页数加到列表l末尾，再次抓取
        except Exception as e:
            fail_urls.append(i)
            print(e)
 
    xiaoqu_pd = pd.DataFrame({"小区名称":xiaoqu_name,"小区编号":xiaoqu_id})
    return xiaoqu_pd

In [ ]:
'''
#抓取小区的名字及编号，然后去重，存入mongo数据库
xiaoqu = get_xiaoqu2()
xiaoqu.drop_duplicates(subset=['小区名称','小区编号'], keep="first", inplace=True)
funds_2_mongo("xiaoqu",xiaoqu)

'''

In [ ]:
#小区具体一页房源信息的抓取，输入为当前页面url,当前爬取页数pa。返回数据为小区房源总数num，该页抓取的房源信息home_list,状态码1或0（1表示成功）
def parse_xiaoqu(url,pa,xiaoquname):
    head = {'Host': 'bj.lianjia.com',
            'Referer': 'https://bj.lianjia.com/chengjiao/',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'
            }
    r = requests.get(url, headers=head,timeout=5)
    html = etree.HTML(r.text)
    datas = html.xpath('//li/div[@class="info"]')
    num = len(datas)
    if num==0:
        return(num,[],0)   #服务器无返回数据，状态码返回0
    house_list=[]
    i=1
    for html1 in datas:
        title = html1.xpath('div[@class="title"]/a/text()')
        
        info = html1.xpath('div[@class="address"]/div[@class="houseInfo"]/text()')
        date = html1.xpath('div[@class="address"]/div[@class="dealDate"]/text()')
        totalprice = html1.xpath('div[@class="address"]/div[@class="totalPrice"]/span/text()')
        
        floor = html1.xpath('div[@class="flood"]/div[@class="positionInfo"]/text()')
        unitprice = html1.xpath('div[@class="flood"]/div[@class="unitPrice"]/span/text()')
        
       
        detailinfo = html1.xpath('div[@class="dealHouseInfo"]/span[@class="dealHouseTxt"]/span/text()')
        #detailinfo_xpath='/html/body/div[5]/div[1]/ul/li[' + str(i) + ']/div/div[4]/span[2]/span[1]/text()'
        #detailinfo = html1.xpath(detailinfo_xpath)
        
        guapaiprice = html1.xpath('div[@class="dealCycleeInfo"]/span[@class="dealCycleTxt"]/span[1]/text()')
        #guapaiprice_xpath='/html/body/div[5]/div[1]/ul/li[' + str(i) + ']/div/div[5]/span[2]/span[1]/text()'
        #guapaiprice = html1.xpath(guapaiprice_xpath)
        
        dealcycle = html1.xpath('div[@class="dealCycleeInfo"]/span[@class="dealCycleTxt"]/span[2]/text()')
        #dealcycle_xpath = '/html/body/div[5]/div[1]/ul/li[' + str(i) + ']/div/div[5]/span[2]/span[2]/text()'
        #dealcycle =  html1.xpath(dealcycle_xpath)
        
        if len(info)==0:
            info = ["暂无朝向信息"]
        if len(date)==0:
            date = ["暂无成交日期"]
        if len(totalprice)==0:
            totalprice = ["暂无成交总价"]
        
        if len(floor)==0:
            floor = ["暂无楼层信息"]
        if len(unitprice)==0:
            unitprice = ["暂无单价信息"]
            
        if len(detailinfo)==0:
            detailinfo = ["暂无满几信息"]
        
        if len(guapaiprice)==0:
            guapaiprice = ["暂无挂牌信息"]
        if len(dealcycle)==0:
            dealcycle = ["暂无交易周期"]
        #info[0] = info[0].replace('\xa0','')  #该条信息中有个html语言的空格符号&nbsp；需要去掉，不然gbk编码会报错，gb18030显示问号
        
        i = i + 1
        a = [xiaoquname, title[0], info[0], date[0], totalprice[0], floor[0], unitprice[0], detailinfo[0], guapaiprice[0], dealcycle[0]]
        house_list.append(a)
        #print(xiaoquname, title[0], info[0], date[0], totalprice[0], floor[0], unitprice[0], detailinfo[0], guapaiprice[0], dealcycle[0])

    print('                *********************         ','第%d页完成！'%pa)
    return (num,house_list,1)


In [ ]:
def crow_xiaoqu(xiaoqu_pd,thread_number):
    
    head = {'Host': 'bj.lianjia.com',
            'Referer': 'https://bj.lianjia.com/chengjiao/',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'
    }
    all_count = len(xiaoqu_pd)
    count=1
    
    pages = 0
    
    global  host_info
    global  host_index
    global  headers
    
    for index,row in xiaoqu_pd.iterrows(): 
        uid=row['小区编号']
        name = row['小区名称']
        url='https://bj.lianjia.com/chengjiao/c%d/'%int(uid)
        
        print(url)
        #h_list=[]      #保存该小区抓取的所有房源信息
        fail_list=[]   #保存第一次抓取失败的页数，第一遍抓取完成后对这些页数再次抓取

        res = requests.get(url=url,headers=head)
        content=etree.HTML(res.text)
        xq_host_count=content.xpath('//div[@class="total fl"]/span/text()')[0]
        xq_host_count=int(xq_host_count)     
        
        # 确定当前小区房源页数pages
        if xq_host_count == 0:
            print("当前线程编号: %d , 所有小区数量: %d , 当前小区序号: %d , 完成抓取比率: %f %% , 小区名称: %s , 小区编号: %s" %(thread_number, all_count,count,float(count/all_count*100),row['小区名称'],row['小区编号']))
            count = count + 1
            continue
        if xq_host_count > 30:
            if xq_host_count % 30 == 0:
                pages = xq_host_count // 30
            else:
                pages = xq_host_count // 30 + 1
   
        for pa in range(1,pages+1):
            new_url = 'https://bj.lianjia.com/chengjiao/pg'+str(pa)+'c'+str(uid)
            try:
                result=parse_xiaoqu(new_url,pa,name)
                status=result[2]
                if status==1:
                    new_list=result[1]
                    #加上锁，多线程将抓取的结果存入全局的pd中
                    lock.acquire()
                    for data in new_list:
                        host_info.loc[host_index]=data
                        host_index = host_index + 1       
                    lock.release()
                    
                    print("小区名称: %s , 小区编号: %s ,小区房屋页数: %d , 当前抓取页数: %d , 完成抓取比率: %f %% "  %(name,uid,pages,pa,float(pa/pages*100)))
                else:
                    fail_list.append(pa)
            except Exception as e:
                fail_list.append(pa)
                print(e)
        
        print('   开始抓取第一次失败页面')
        for pa in fail_list:
            new_url = 'https://bj.lianjia.com/chengjiao/pg' + str(pa) + 'c' + str(uid)
            print(new_url)
            try:
                result = parse_xiaoqu(new_url,pa,name)
                status = result[2]
                if status == 1:
                    new_list = result[1]
                    lock.acquire()
                    for data in new_list:
                        host_info.loc[host_index]=data
                        host_index = host_index + 1 
                    lock.release()
                    
                    print("小区名称: %s , 小区编号: %s ,小区房屋页数: %d , 当前抓取页数: %d , 该小区完成抓取比率: %f %% "  %(name,uid,pages,pa,float(pa/pages*100)))
                else:
                    pass
            except Exception as e:
                print(e)

        #将一个小区的房屋信息，存放到pd中
        print("################################################################")
        print("抓取的数据长度： %d , 分配的pd的index: %d " %(len(host_info),host_index))
               
        
        print("当前线程编号: %d , 所有小区数量: %d , 当前小区序号: %d , 完成抓取比率: %f %% , 小区名称: %s , 小区编号: %s" %(thread_number, all_count,count,float(count/all_count*100),row['小区名称'],row['小区编号']))
        count = count + 1

In [ ]:
#读取小区信息
xiaoqus = pd.DataFrame(columns=["小区名称","小区编号"])
xiaoqus = funds_f_mongo("xiaoqu")
all_count=len(xiaoqus)

In [ ]:
#读取房屋信息
host_info = funds_f_mongo("bj_host")
host_index = len(host_info)

In [ ]:
all_xiaoqus = xiaoqus[1200:]

In [ ]:
count = int(len(all_xiaoqus)/4)

In [ ]:
print(len(all_xiaoqus))
print(count)

In [ ]:
all_xiaoqus

In [ ]:
xiaoqu_part1=all_xiaoqus.iloc[:count]
xiaoqu_part2=all_xiaoqus.iloc[count:count*2]
xiaoqu_part3=all_xiaoqus.iloc[count*2:count*3]
xiaoqu_part4=all_xiaoqus.iloc[count*3:]

In [ ]:
print("开始多线程抓取时间：")
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
t1 = threading.Thread(target=crow_xiaoqu , args=(xiaoqu_part1,1))
t1.start()
t2 = threading.Thread(target=crow_xiaoqu , args=(xiaoqu_part2,2))
t2.start()
t3 = threading.Thread(target=crow_xiaoqu , args=(xiaoqu_part3,3))
t3.start()
t4 = threading.Thread(target=crow_xiaoqu , args=(xiaoqu_part4,4))
t4.start()
t1.join()
t2.join()
t3.join()
t4.join()
print("结束多线程抓取时间：")
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

In [ ]:
funds_2_mongo("bj_host",host_info)